# Narrative Scene Understanding - Basic Usage

This notebook demonstrates the basic usage of the Narrative Scene Understanding system.

In [1]:
# Import required modules
import sys
import os
import json
from pathlib import Path

# Add parent directory to path to import from modules
sys.path.append(os.path.abspath(os.path.join('..', '..')))

# Import core functionality
from narrative_scene_understanding import process_video, query_scene
from modules.utils import visualize_graph

## 1. Configuration

First, let's set up the configuration for our analysis.

In [2]:
# Define the configuration
config = {
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "output_dir": "../outputs/notebook_output",
    "frame_rate": 2.0,  # Extract 2 frames per second
    "adaptive_sampling": True,  # Use adaptive sampling based on motion
    "max_frames": 300,  # Limit for demonstration
    # Configure LLM to use Ollama
    "query_engine": {
        "use_ollama": True,
        "ollama_model": "llama3",  # Or any other model you have in Ollama
        "ollama_url": "http://localhost:11434",
        "temperature": 0.2
    }
}

# Create output directory
os.makedirs(config["output_dir"], exist_ok=True)

# Path to the video file
video_path = "../videos/movie_scene.mp4"

# Check if video exists
if not os.path.exists(video_path):
    print(f"Video file not found: {video_path}")
    print("Please download example videos or update the path to your video file.")

## 2. Process the Video

Now, let's process the video to extract narrative understanding.

In [3]:
# Process the video
results = process_video(video_path, config)

# Print the summary
print("\n===== SCENE SUMMARY =====")
print(results["summary"])

## 3. Visualize the Knowledge Graph

Let's visualize the knowledge graph to better understand the scene structure.

In [4]:
# Visualize the graph
graph = results["graph"]
vis_path = os.path.join(config["output_dir"], "graph_visualization.png")
visualize_graph(graph, vis_path)

# Display the image
from IPython.display import Image
Image(filename=vis_path)

## 4. Query the Scene

Now, let's ask some questions about the scene.

In [5]:
# Define a question
question = "Why does Character A appear nervous during the conversation?"

# Query the scene
response = query_scene(question, results=results)

# Print the answer
print("\n===== QUESTION =====")
print(question)
print("\n===== ANSWER =====")
print(response["answer"])
print("\n===== REASONING PROCESS =====")
print(response["reasoning"])

## 5. Explore Character Emotions

Let's analyze the emotional arcs of characters in the scene.

In [6]:
import matplotlib.pyplot as plt
import numpy as np

# Emotion to valence mapping (same as in the system)
EMOTION_VALENCE = {
    "happy": 1.0, "content": 0.6, "neutral": 0.0, "concerned": -0.4,
    "nervous": -0.5, "sad": -0.6, "angry": -0.8, "surprised": 0.3,
    "distressed": -0.7, "confused": -0.3
}

def emotion_to_valence(emotion):
    emotion = emotion.lower()
    if emotion in EMOTION_VALENCE:
        return EMOTION_VALENCE[emotion]
    # Try to find partial match
    for key, value in EMOTION_VALENCE.items():
        if key in emotion or emotion in key:
            return value
    return 0.0

# Create figure
plt.figure(figsize=(10, 6))

# Plot emotional arcs for each character
for char_id, char_info in results["analysis"]["characters"].items():
    char_name = char_info.get("name", char_id)
    emotions = char_info.get("emotions", [])
    
    if emotions:
        timestamps = [e["timestamp"] for e in emotions]
        valence = [emotion_to_valence(e["emotion"]) * e.get("intensity", 1.0) for e in emotions]
        
        plt.plot(timestamps, valence, 'o-', label=char_name)
        
        # Annotate emotions
        for t, v, e in zip(timestamps, valence, emotions):
            plt.annotate(e["emotion"], (t, v), textcoords="offset points", xytext=(0, 10), ha='center')

# Set up plot
plt.axhline(y=0, color='gray', linestyle='--', alpha=0.5)  # Neutral line
plt.xlabel("Time (seconds)")
plt.ylabel("Emotional Valence")
plt.title("Character Emotional Arcs")
plt.grid(True, alpha=0.3)
plt.ylim(-1.1, 1.1)
plt.legend()
plt.tight_layout()

# Save and display
plt.savefig(os.path.join(config["output_dir"], "emotional_arcs.png"))
plt.show()

## 6. Save Results

Finally, let's save the full results to a JSON file for further analysis.

In [7]:
# Save results to JSON
output_path = os.path.join(config["output_dir"], "full_analysis.json")

# Convert the NetworkX graph to a serializable format
serializable_results = results.copy()
serializable_results["graph"] = {
    "nodes": [
        {"id": node, **{k: v for k, v in attrs.items() if isinstance(v, (str, int, float, bool, list, dict))}}
        for node, attrs in graph.nodes(data=True)
    ],
    "edges": [
        {"source": u, "target": v, **{k: v for k, v in attrs.items() if isinstance(v, (str, int, float, bool, list, dict))}}
        for u, v, attrs in graph.edges(data=True)
    ]
}

with open(output_path, 'w') as f:
    json.dump(serializable_results, f, indent=2)

print(f"Results saved to {output_path}")